<a href="https://colab.research.google.com/github/hyun-tae04/school_project_python/blob/main/6%EC%A3%BC%EC%B0%A8%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gensim

In [2]:
!pip install summarizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for summarizer: filename=summarizer-0.0.7-py2.py3-none-any.whl size=284209 sha256=03a53c827c451d4d7ca1457f60312e3aa04973c5c81dc71401eb280d254e7895
  Stored in directory: /root/.cache/pip/wheels/20/bb/2d/1fe057c2f729818a5f28c312c3667e8b9d5cfd4af4a39895e7
Successfully built summarizer


In [3]:
!pip install summarization

  Preparing metadata (setup.py) ... done
  Created wheel for summarization: filename=summarization-0.0.1-py3-none-any.whl size=1410 sha256=27dbe95bc1bbb3c0f31b4807d3e099a5f356078c9d4b308545e60ef4e4251eb7
  Stored in directory: /root/.cache/pip/wheels/13/67/59/2b8668bab99bd69e2c9fc52792d4bd94445b9a9428a2d50519
Successfully built summarization


In [7]:
!pip install gensim==3.8.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 40.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gensim: filename=gensim-3.8.0-cp310-cp310-linux_x86_64.whl size=24681053 sha256=cc1255846e8005a7990cf6d0f591d68331e0191835a4f3f44c616efc3601a92e
  Stored in directory: /root/.cache/pip/wheels/01/ab/bf/07d4e3462e9f8d17d98c2fb5e870d7e9934c8482a32fc3ef41
Successfully built gensim
  Attempting uninstall: gensim
    Found existing installation: gensim 4.3.0
    Uninstalling gensim-4.3.0:
      Successfully uninstalled gensim-4.3.0


In [8]:
!pip show gensim

Name: gensim
Version: 3.8.0
Summary: Python framework for fast Vector Space Modelling
Home-page: http://radimrehurek.com/gensim
Author: Radim Rehurek
Author-email: me@radimrehurek.com
License: LGPLv2.1
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, scipy, six, smart-open
Required-by: 


In [9]:
from gensim.summarization.summarizer import summarize

In [10]:
import requests
from bs4 import BeautifulSoup
import time
import re
from tqdm import tqdm

# 페이지 번호 생성하는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num + 1
    else:
        return num + 9 * (num - 1)

# 검색 URL 생성하는 함수
def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
        print("생성된 URL: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
            urls.append(url)
        print("생성된 URLs: ", urls)
        return urls

def articles_crawler(url):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}
    original_html = requests.get(url, headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")
    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = [i.attrs['href'] for i in url_naver]
    return url


# 검색어와 페이지 입력 받기
search = input("검색어를 입력하세요:")
start_page = int(input("\n크롤링을 시작할 페이지를 :"))
end_page = int(input("\n크롤링을 종료할 페이지를 :"))

# Naver 검색 URL 생성
urls = makeUrl(search, start_page, end_page)

# 기사 URL 크롤링
article_urls = []
for url in urls:
    article_urls.extend(articles_crawler(url))

# Naver 뉴스 URL 필터링
final_urls = [url for url in article_urls if "news.naver.com" in url]

# 데이터 저장을 위한 리스트 초기화
news_titles = []
news_contents = []

# 뉴스 내용 추출하는 함수
def extract_news_content(url):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}
    news = requests.get(url, headers=headers)
    news_html = BeautifulSoup(news.text, "html.parser")
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title is None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    content = news_html.select("article#dic_area")
    if not content:
        content = news_html.select("#articleBody")
    title = re.sub(pattern='<[^>]*>', repl='', string=str(title))
    content = re.sub(pattern='<[^>]*>', repl='', string=''.join(map(str, content)))
    content = content.replace("""[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}""", '')
    return title, content

# 뉴스 내용 추출
for url in tqdm(final_urls):
    title, content = extract_news_content(url)
    news_contents.append(content)

text = ' '
for i in news_contents:
    text += i

print(text)



검색어를 입력하세요:코로나

크롤링을 시작할 페이지를 :1

크롤링을 종료할 페이지를 :2
생성된 URLs:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=코로나&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=코로나&start=11']


100%|██████████| 10/10 [00:07<00:00,  1.26it/s]

 
12~64세 일반인은 11월1일부터 접종



[서울=뉴시스]질병관리청(질병청)은 지난달 26일 이같은 내용을 담은 '2023~2024절기 코로나19 예방접종 추진계획'을 발표했다. (사진제공=질병청) 2023.09.26. photo@newsis.com. *재판매 및 DB 금지[서울=뉴시스]권지원 기자 = 19일부터 고위험군 대상으로 코로나19 백신 접종이 시작된다. 65세 이상 고령층은 이날부터 코로나19 백신과 함께 인플루엔자(독감) 백신도 동시 접종받을 수 있다.질병관리청에 따르면 방역 당국은 '2023∼2024절기 코로나19 예방접종 추진계획'에 따라 이날부터 내년 3월31일까지 동절기 전 국민 대상 무료 접종을 실시한다.이날부터 65세 이상 고령자와 12~64세 면역저하자, 감염취약시설 입소자나 종사자는 우선적으로 코로나19 백신 접종이 가능하다. 그 외 고위험군이 아닌 12~64세 일반인은 다음 달 1일부터 코로나19 백신 접종을 받을 수 있다. 사전 예약은 지난 18일에 시작됐다.이번 접종에 활용하는 백신은 현재 유행하는 XBB계열 변이에 대응해 화이자·모더나사가 개발한 XBB.1.5 대응 단가백신이다. 다만 mRNA 백신의 금기 또는 기피자는 추후에 연내 도입 예정인 유전자 재조합 백신인 노바백스 XBB.1.5 단가 백신으로 접종할 수 있다.인플루엔자 백신접종과 유사하게, 이전 접종력과 관계없이 접종 기간 내 1회 접종으로 접종이 완료된다. 그동안 기초 접종을 완료하지 않았더라도 신규백신을 접종했다면, 추가적인 접종은 필요하지 않다.접종은 마지막 접종일로부터 3개월(90일) 이후 가능하다. 접종은 전국 위탁의료기관과 보건소에서 가능하다.한편 당국은 이번 동절기 접종부터는 독감 백신과 동시 접종을 권고하고 있다. 당국은 최근 국내외 연구를 통해 코로나19와 독감 백신 동시 접종 시 면역 형성과 안전에 문제가 없음을 확인했다고 밝혔다. 지난 2022~2023년 절기 65세 이상 중 코로나19와 인플루엔자 백신 동시접종자 24만 명의 이상 반응 신

In [11]:
print(summarize(text))

[서울=뉴시스]질병관리청(질병청)은 지난달 26일 이같은 내용을 담은 '2023~2024절기 코로나19 예방접종 추진계획'을 발표했다.
*재판매 및 DB 금지[서울=뉴시스]권지원 기자 = 19일부터 고위험군 대상으로 코로나19 백신 접종이 시작된다.
65세 이상 고령층은 이날부터 코로나19 백신과 함께 인플루엔자(독감) 백신도 동시 접종받을 수 있다.질병관리청에 따르면 방역 당국은 '2023∼2024절기 코로나19 예방접종 추진계획'에 따라 이날부터 내년 3월31일까지 동절기 전 국민 대상 무료 접종을 실시한다.이날부터 65세 이상 고령자와 12~64세 면역저하자, 감염취약시설 입소자나 종사자는 우선적으로 코로나19 백신 접종이 가능하다.
그 외 고위험군이 아닌 12~64세 일반인은 다음 달 1일부터 코로나19 백신 접종을 받을 수 있다.
사전 예약은 지난 18일에 시작됐다.이번 접종에 활용하는 백신은 현재 유행하는 XBB계열 변이에 대응해 화이자·모더나사가 개발한 XBB.1.5 대응 단가백신이다.
다만 mRNA 백신의 금기 또는 기피자는 추후에 연내 도입 예정인 유전자 재조합 백신인 노바백스 XBB.1.5 단가 백신으로 접종할 수 있다.인플루엔자 백신접종과 유사하게, 이전 접종력과 관계없이 접종 기간 내 1회 접종으로 접종이 완료된다.
접종은 전국 위탁의료기관과 보건소에서 가능하다.한편 당국은 이번 동절기 접종부터는 독감 백신과 동시 접종을 권고하고 있다.
지난 2022~2023년 절기 65세 이상 중 코로나19와 인플루엔자 백신 동시접종자 24만 명의 이상 반응 신고율(0.04%)은 코로나19 백신 단독접종자(880만 명)의 이상 반응 신고율(0.07%)보다 40% 낮은 수준으로 확인됐다.미국 질병예방센터(CDC)도 별도의 추가 방문 없이 두 백신을 동시에 접종하는 장점이 크다고 분석하면서 백신의 동시접종을 권고하고 있다．
19일부터 코로나19 백신 접종이 시작된다.
김우주 고려대구로병원 감염내과 교수(왼쪽)는 “65세 이상, 요양시설 등 감염 취약 시설 

In [12]:
print(summarize(text, ratio=0.1))

*재판매 및 DB 금지[서울=뉴시스]권지원 기자 = 19일부터 고위험군 대상으로 코로나19 백신 접종이 시작된다.
65세 이상 고령층은 이날부터 코로나19 백신과 함께 인플루엔자(독감) 백신도 동시 접종받을 수 있다.질병관리청에 따르면 방역 당국은 '2023∼2024절기 코로나19 예방접종 추진계획'에 따라 이날부터 내년 3월31일까지 동절기 전 국민 대상 무료 접종을 실시한다.이날부터 65세 이상 고령자와 12~64세 면역저하자, 감염취약시설 입소자나 종사자는 우선적으로 코로나19 백신 접종이 가능하다.
그 외 고위험군이 아닌 12~64세 일반인은 다음 달 1일부터 코로나19 백신 접종을 받을 수 있다.
접종은 전국 위탁의료기관과 보건소에서 가능하다.한편 당국은 이번 동절기 접종부터는 독감 백신과 동시 접종을 권고하고 있다.
지난 2022~2023년 절기 65세 이상 중 코로나19와 인플루엔자 백신 동시접종자 24만 명의 이상 반응 신고율(0.04%)은 코로나19 백신 단독접종자(880만 명)의 이상 반응 신고율(0.07%)보다 40% 낮은 수준으로 확인됐다.미국 질병예방센터(CDC)도 별도의 추가 방문 없이 두 백신을 동시에 접종하는 장점이 크다고 분석하면서 백신의 동시접종을 권고하고 있다．
고위험군에 해당하는 만성질환자와 임신부는 독감과 코로나19 예방접종은 매우 중요하므로 가급적 접종을 받는 것이 좋다.”  ―최근엔 EG.5(에리스) 등 신규 변이가 등장했다는데 기존 백신으로 예방할 수 있나.
실제로 질병관리청에서 발표한 자료에 따르면, 모더나 백신은 최근 유행하는 주요 변이 EG.5(에리스)에도 중화항체 증가율이 10.7배 되는 것으로 나타났다.
또 코로나19 백신과 독감 백신을 동시 접종했을 때 각각 생긴 항체가 서로 간 백신 효과를 감소시키지 않는 것으로 나타났다.
따라서 코로나19 백신과 독감 백신 동시 접종은 백신의 안전성과 효과 확보뿐만 아니라 백신 접종 시간도 절약할 수 있다.”  ―국민들이 여전히 백신 부작용에 대해 민감하다.
정부는 국내

In [13]:
print(summarize(text, ratio=0.05))

*재판매 및 DB 금지[서울=뉴시스]권지원 기자 = 19일부터 고위험군 대상으로 코로나19 백신 접종이 시작된다.
65세 이상 고령층은 이날부터 코로나19 백신과 함께 인플루엔자(독감) 백신도 동시 접종받을 수 있다.질병관리청에 따르면 방역 당국은 '2023∼2024절기 코로나19 예방접종 추진계획'에 따라 이날부터 내년 3월31일까지 동절기 전 국민 대상 무료 접종을 실시한다.이날부터 65세 이상 고령자와 12~64세 면역저하자, 감염취약시설 입소자나 종사자는 우선적으로 코로나19 백신 접종이 가능하다.
지난 2022~2023년 절기 65세 이상 중 코로나19와 인플루엔자 백신 동시접종자 24만 명의 이상 반응 신고율(0.04%)은 코로나19 백신 단독접종자(880만 명)의 이상 반응 신고율(0.07%)보다 40% 낮은 수준으로 확인됐다.미국 질병예방센터(CDC)도 별도의 추가 방문 없이 두 백신을 동시에 접종하는 장점이 크다고 분석하면서 백신의 동시접종을 권고하고 있다．
또 코로나19 백신과 독감 백신을 동시 접종했을 때 각각 생긴 항체가 서로 간 백신 효과를 감소시키지 않는 것으로 나타났다.
따라서 코로나19 백신과 독감 백신 동시 접종은 백신의 안전성과 효과 확보뿐만 아니라 백신 접종 시간도 절약할 수 있다.”  ―국민들이 여전히 백신 부작용에 대해 민감하다.
질병관리청 제공오는 19일부터 최근 유행 중인 코로나19 XBB계열 변이에 대응 가능한 단가백신 접종이 시작된다.
이에 질병관리청은 지난 16일 충북 오송 질병청 국립의과학지식센터에서 출입기자단을 대상으로 한 감염병 예방관리 아카데미를 열고 '2023~24절기 코로나19 예방접종'의 효용성과 중요성을 거듭 강조했다.
Q. 동절기 코로나19 예방접종 일정이 어떻게 되나.A: 최우선 접종순위인 △65세 이상 △12~64세 면역저하자 △감염취약시설 구성원(요양병원·시설 입소자, 종사자 등) 등은 19일부터 접종에 들어간다.
코로나19를 앓아봤고 수차례 반복 접종을 했는데도, 신규 접종이 필요한가.A: 

In [14]:
print(summarize(text, word_count=300))

*재판매 및 DB 금지[서울=뉴시스]권지원 기자 = 19일부터 고위험군 대상으로 코로나19 백신 접종이 시작된다.
65세 이상 고령층은 이날부터 코로나19 백신과 함께 인플루엔자(독감) 백신도 동시 접종받을 수 있다.질병관리청에 따르면 방역 당국은 '2023∼2024절기 코로나19 예방접종 추진계획'에 따라 이날부터 내년 3월31일까지 동절기 전 국민 대상 무료 접종을 실시한다.이날부터 65세 이상 고령자와 12~64세 면역저하자, 감염취약시설 입소자나 종사자는 우선적으로 코로나19 백신 접종이 가능하다.
그 외 고위험군이 아닌 12~64세 일반인은 다음 달 1일부터 코로나19 백신 접종을 받을 수 있다.
지난 2022~2023년 절기 65세 이상 중 코로나19와 인플루엔자 백신 동시접종자 24만 명의 이상 반응 신고율(0.04%)은 코로나19 백신 단독접종자(880만 명)의 이상 반응 신고율(0.07%)보다 40% 낮은 수준으로 확인됐다.미국 질병예방센터(CDC)도 별도의 추가 방문 없이 두 백신을 동시에 접종하는 장점이 크다고 분석하면서 백신의 동시접종을 권고하고 있다．
고위험군에 해당하는 만성질환자와 임신부는 독감과 코로나19 예방접종은 매우 중요하므로 가급적 접종을 받는 것이 좋다.”  ―최근엔 EG.5(에리스) 등 신규 변이가 등장했다는데 기존 백신으로 예방할 수 있나.
또 코로나19 백신과 독감 백신을 동시 접종했을 때 각각 생긴 항체가 서로 간 백신 효과를 감소시키지 않는 것으로 나타났다.
따라서 코로나19 백신과 독감 백신 동시 접종은 백신의 안전성과 효과 확보뿐만 아니라 백신 접종 시간도 절약할 수 있다.”  ―국민들이 여전히 백신 부작용에 대해 민감하다.
정부는 국내 건강보험심사평가원 빅데이터를 활용해 고령자, 만성질환자, 면역저하자 및 임신부 등 각각의 고위험군에서 코로나19 예방접종의 중증, 입원 및 사망 예방 효과를 구체적으로 발표해 국민이 예방접종의 필요성을 느낄 수 있도록 할 필요가 있다.”
질병관리청 제공오는 19일부터 최근 유행 

In [15]:
print(summarize(text, ratio=0.01))

65세 이상 고령층은 이날부터 코로나19 백신과 함께 인플루엔자(독감) 백신도 동시 접종받을 수 있다.질병관리청에 따르면 방역 당국은 '2023∼2024절기 코로나19 예방접종 추진계획'에 따라 이날부터 내년 3월31일까지 동절기 전 국민 대상 무료 접종을 실시한다.이날부터 65세 이상 고령자와 12~64세 면역저하자, 감염취약시설 입소자나 종사자는 우선적으로 코로나19 백신 접종이 가능하다.
